In [1]:
import pandas as pd 
import numpy as np
from network_coverage_api.app.utils import convert_lambert93_df_to_wgs84, calculate_tolerance

In [2]:
df = pd.read_csv('../data/2018_01_Sites_mobiles_2G_3G_4G_France_metropolitaine_L93.csv', sep=';')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77148 entries, 0 to 77147
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Operateur  77148 non-null  int64  
 1   x          77147 non-null  float64
 2   y          77147 non-null  float64
 3   2G         77148 non-null  int64  
 4   3G         77148 non-null  int64  
 5   4G         77148 non-null  int64  
dtypes: float64(2), int64(4)
memory usage: 3.5 MB


In [4]:
df.rename(columns={'Operateur': 'operateur'}, inplace=True)

In [5]:
df.head()

,operateur,x,y,2G,3G,4G
0,20801,102980.0,6847973.0,1,1,0
1,20810,103113.0,6848661.0,1,1,0
2,20820,103114.0,6848664.0,1,1,1
3,20801,112032.0,6840427.0,0,1,1
4,20801,115635.0,6799938.0,1,1,0


In [6]:
df.rename(columns={'Operateur': 'operateur'}, inplace=True)

#### Create a copy for processing 

In [7]:
df_cleaned = df.copy()

#### Check for duplicates entries

In [8]:
duplicates = df_cleaned[df_cleaned.duplicated(subset=['operateur', 'x', 'y', '2G', '3G', '4G'], keep=False)]
duplicates

,operateur,x,y,2G,3G,4G
5736,20801,373594.0,6298494.0,0,1,0
5737,20801,373594.0,6298494.0,0,1,0
7081,20801,397239.0,6857986.0,0,0,0
7082,20801,397239.0,6857986.0,0,0,0
8577,20801,418409.0,6428567.0,1,1,1
...,...,...,...,...,...,...
74733,20820,1037869.0,6298619.0,1,1,1
74975,20820,1041366.0,6296402.0,1,1,1
74976,20820,1041366.0,6296402.0,1,1,1
76524,20815,1180107.0,6091654.0,0,1,1


In [9]:
print(f'Records before deleting duplicates: {len(df_cleaned)}')
df_cleaned = df_cleaned.drop_duplicates(subset=['operateur', 'x', 'y', '2G', '3G', '4G'], keep='first')
print(f'Records after deleting duplicates: {len(df_cleaned)}')

Records before deleting duplicates: 77148
Records after deleting duplicates: 77024


#### Operators codes mapping

In [10]:
unique_operateurs = df_cleaned['operateur'].unique()
unique_operateurs

array([20801, 20810, 20820, 20815])

In [11]:
operators_mapping = {
    20801: 'Orange',
    20810: 'SFR',
    20815: 'Free', 
    20820: 'Bouygue'
}

df_cleaned['operator_name'] = df_cleaned['operateur'].map(operators_mapping)

#### Transform to GPS Coordinates 

In [12]:
df_cleaned = convert_lambert93_df_to_wgs84(df_cleaned)

/Users/eo/src/network_coverage_api/network_coverage_api/app/utils.py:60: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  dataframe["longitude"], dataframe["latitude"] = pyproj.transform(


In [13]:
df_cleaned.describe()

,operateur,x,y,2G,3G,4G,longitude,latitude
count,77024.000000,7.702300e+04,7.702300e+04,77024.000000,77024.000000,77024.000000,77023.000000,77023.000000
mean,20810.365419,6.952889e+05,6.637132e+06,0.746611,0.938422,0.658587,2.918664,46.801993
std,7.349988,2.068679e+05,2.504402e+05,0.434954,0.240389,0.474187,2.697567,2.258747
min,20801.000000,1.029800e+05,6.050021e+06,0.000000,0.000000,0.000000,-5.088856,41.364455
25%,20801.000000,5.697050e+05,6.422107e+06,0.000000,1.000000,0.000000,1.333208,44.855818
50%,20810.000000,6.744140e+05,6.689987e+06,1.000000,1.000000,1.000000,2.659307,47.258051
75%,20815.000000,8.546840e+05,6.859646e+06,1.000000,1.000000,1.000000,5.005605,48.827163
max,20820.000000,1.240585e+06,7.113682e+06,1.000000,1.000000,1.000000,9.550389,51.106451


In [14]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77024 entries, 0 to 77147
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   operateur      77024 non-null  int64  
 1   x              77023 non-null  float64
 2   y              77023 non-null  float64
 3   2G             77024 non-null  int64  
 4   3G             77024 non-null  int64  
 5   4G             77024 non-null  int64  
 6   operator_name  77024 non-null  object 
 7   longitude      77023 non-null  float64
 8   latitude       77023 non-null  float64
dtypes: float64(4), int64(4), object(1)
memory usage: 5.9+ MB


### Find Tolerance 

In [15]:
LATITUDE_PARIS = 48.860248 
TOLERANCE_IN_METERS = 500
tolerance = calculate_tolerance(LATITUDE_PARIS, TOLERANCE_IN_METERS)
tolerance = round(tolerance, 4)
tolerance

0.0082

In [16]:
#longitude = 2.380383
#latitude = 48.860248
#longitude = 2.341185
#latitude = 48.850774
#longitude = 2.27666
#latitude = 48.860812
longitude = -0.449476
latitude = 46.326966


matches_within_tolerance = df_cleaned[
        (np.isclose(df_cleaned.longitude, longitude, atol=tolerance))
        & (np.isclose(df_cleaned.latitude, latitude, atol=tolerance))
    ]

matches_within_tolerance

,operateur,x,y,2G,3G,4G,operator_name,longitude,latitude
9883,20801,434064.0,6586337.0,1,1,1,Orange,-0.457668,46.324532
9884,20801,434064.0,6586337.0,1,0,0,Orange,-0.457668,46.324532
9885,20820,434115.0,6585703.0,1,1,1,Bouygue,-0.456645,46.318849
9886,20810,434115.0,6585703.0,1,1,0,SFR,-0.456645,46.318849
9894,20820,434166.0,6587256.0,1,1,1,Bouygue,-0.456866,46.332840
9900,20801,434189.0,6587422.0,1,1,1,Orange,-0.456662,46.334342


In [17]:
import numpy as np

target_x = 654540.03
target_y = 6862424.29

filtered_df = df_cleaned[
    (np.isclose(df_cleaned['x'], target_x, atol=TOLERANCE_IN_METERS)) &
    (np.isclose(df_cleaned['y'], target_y, atol=TOLERANCE_IN_METERS))
]

print(filtered_df)

       operateur         x          y  2G  3G  4G operator_name  longitude  \
33683      20820  654078.0  6862405.0   1   1   1       Bouygue   2.374088   
33687      20801  654087.0  6862417.0   1   1   1        Orange   2.374209   
33700      20820  654100.0  6862969.0   1   1   1       Bouygue   2.374327   
33710      20820  654107.0  6862503.0   1   0   0       Bouygue   2.374473   
33722      20810  654118.0  6861902.0   0   0   1           SFR   2.374687   
33726      20801  654125.0  6862531.0   0   0   0        Orange   2.374715   
33728      20810  654134.0  6862906.0   1   1   1           SFR   2.374797   
33800      20801  654278.0  6862619.0   0   1   0        Orange   2.376791   
33801      20801  654279.0  6862196.0   0   1   0        Orange   2.376850   
33816      20810  654308.0  6862174.0   0   1   1           SFR   2.377247   
33824      20815  654319.0  6862176.0   0   1   1          Free   2.377397   
33831      20810  654331.0  6862696.0   1   1   1           SFR 

In [18]:
df_cleaned.to_csv('../data/network_data_cleaned.csv')